# ipywidgets

In [1]:
from __future__ import print_function
import sys
import pandas as pd
from io import StringIO
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [2]:
out = widgets.Output(layout={'border': '1px solid black'})
up = widgets.FileUpload(accept="", multiple=False)

In [3]:
eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)

rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

In [4]:
button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')
accordion = widgets.Accordion(children=[up])
accordion.set_title(0, 'File Selection')
accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview]),
    out
])

In [5]:
def content_parser():
    if up.value == {}:
        with out:
            print('No CSV loaded')    
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
        if typ == "application/vnd.ms-excel":
            content = up_value[i]["content"]
            content_str = str(content, 'utf-8')
    
            if eraser.value != {}: 
                for val in eraser.value:
                    if val == "tab":
                        content_str = content_str.replace("\t","")
                    else:
                        content_str = content_str.replace(val,"")
            if content_str != "":
                str_io = StringIO(content_str)
                return str_io


def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(content, index_col=False, skiprows=rows.value)    
        return df
    else:
        return None


def preview():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n -----Now this is how your DF looks like:----- \n')
        if df is not None:
            print(df.head(10))
        else:
            print('Configuration is wrong/missing...')
                
def preview_clicked(b):
    preview()

In [6]:
button_preview.on_click(preview_clicked)

In [7]:
accordion_box

In [8]:
# up.value

In [9]:
from IPython.display import HTML

HTML('''
<!-- Button -->
<form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
</form>

<!-- Script -->
<script>

  // Replace global?
  var code_show = true;

  /**
   * Simple function for toggle button.
   *
   * @param {void} - Nothing.
   * @return {void} - Nothing.
   */
  function code_toggle() {

      if (code_show){
          $('div.input').hide();
      } else {
          $('div.input').show();
      }

      // Set boolean to toggle back
      code_show = !code_show;
  }
    
  $( document ).ready(code_toggle);
</script>
''')